In [1]:
import numpy as np
import time
import pandas as pd



In [7]:
from permutationTest import coinShiftInt,coinShiftFloat, GreenInt, GreenFloat, GreenFloatCuda,GreenIntCuda


In [8]:
np.random.seed(10)
n = 200
n_samples = 5
A = np.asarray([np.random.randint(0,n,n,dtype=np.int32) for _ in range(n_samples)])
B = np.asarray([np.random.randint(0,n,n,dtype=np.int32) for _ in range(n_samples)])

In [9]:
%%time 

print(GreenIntCuda(A, B))


[0.22811788 0.91310021 0.75868212 0.63274077 0.70694501]
CPU times: user 481 ms, sys: 88.3 ms, total: 570 ms
Wall time: 585 ms


In [10]:
%%time 

print(GreenInt(A, B))


[0.22811788 0.91310021 0.75868212 0.63274077 0.70694501]
CPU times: user 9.02 s, sys: 116 ms, total: 9.13 s
Wall time: 9.1 s


In [11]:
%%time 

print(GreenInt(A, B,multiThread=True))

[0.22811788 0.91310021 0.75868212 0.63274077 0.70694501]
CPU times: user 24.1 s, sys: 139 ms, total: 24.2 s
Wall time: 3.11 s


In [12]:
%%time 

print(GreenFloat(A, B,n))

[0.22811788 0.91310021 0.75868212 0.63274077 0.47043136]
CPU times: user 8.9 s, sys: 120 ms, total: 9.02 s
Wall time: 8.97 s


In [13]:
%%time 
print(GreenFloatCuda(A, B, n))


[0.22811788 0.91310021 0.75868212 0.63274077 0.47043136]
CPU times: user 453 ms, sys: 24 ms, total: 477 ms
Wall time: 475 ms


In [14]:
%%time 
print(coinShiftInt(A, B))


[0.22811788 0.91310021 0.75868212 0.63274077 0.70694501]
CPU times: user 2.84 s, sys: 92.1 ms, total: 2.93 s
Wall time: 2.91 s


In [15]:
%%time 
print(coinShiftFloat(A, B,n))


[0.22811788 0.91310021 0.75868212 0.63274077 0.47043136]
CPU times: user 2.84 s, sys: 84.2 ms, total: 2.92 s
Wall time: 2.91 s


In [ ]:
coinShift(*PreProcessoinShift(A[0], B[0]))

In [ ]:
%time GreenIntCuda(A[0], B[0])

In [ ]:
%time print(GreenIntCuda(A, B))



In [ ]:
from significance_of_mean_cuda import significance_of_mean_cuda

In [ ]:
%%time 
SGM = significance_of_mean_cuda(5,dtype_v=np.uint64,dtype_A=np.float64)
SGM.run(A,B)

In [ ]:
SGM.p_values

In [ ]:
GreenIntDataPreProcess()

In [ ]:
A.shape

In [ ]:
A[0][np.newaxis,:].shape

In [ ]:
import numpy as np
import time
import pandas as pd
import random

In [ ]:
from permutationTestC import GrennPdistOpenMP, GrennPdist, coinShiftPdist

In [ ]:
from permutationTestCuda import greenCUDAMultipleSamples, greenCUDA

In [ ]:
def getDataCoinShift(A,B):
    scores = np.concatenate((A,B))
    n = scores.shape[0]
    m = A.shape[0]

    add = np.min(scores - 1)
    scores = scores - add

    m_b = sum(np.sort(scores)[(n - m) :n])

    score_a = np.ones(n, dtype=np.int32)


    im_a = m
    im_b = m_b

    return score_a,scores,im_a, im_b 

def getDataGreen(A,B):
    x = A
    y = B
    m = x.shape[0]
    n = y.shape[0]
    
    z = np.concatenate((A,B))
    z.sort()
    z = z - min(z)
    S = z[m:].sum()
      
    return z, m, n, S

def get_p(pdist,a):
    p = pdist[(a-min(a)).sum():].sum()
    p = 2 * min( p, (1-p))
    return p

In [ ]:
def coinPy(iscore_a, iscore_b, im_a, im_b):
    s_a = 0
    s_b = 0
    
    sum_a = 0
    sum_b = 0
    msum = 0.0
    size_a = iscore_a.shape[0]
    size_b = iscore_b.shape[0]
    
    n = size_a

    for i in range(n):
        sum_a += iscore_a[i];
        sum_b += iscore_b[i];


    

    sum_a = min(sum_a, im_a);
    sum_b = min(sum_b, im_b);

    dH = np.zeros((sum_a + 1) * (sum_b + 1))

    dH[0] = 1.0;

    for k in range(n):
        s_a += iscore_a[k];
        s_b += iscore_b[k];
        #for i in range(min(im_a, s_a), iscore_a[k]-1,-1):
        for j in range(min(im_b,s_b), iscore_b[k]-1,-1):

            #for j in range(min(im_b,s_b), iscore_b[k]-1,-1):
            for i in shuffle(list(range(min(im_a, s_a), iscore_a[k]-1,-1))):
                isb = i * (sum_b + 1)
                dH[isb + j] += dH[(i - iscore_a[k]) * (sum_b + 1) + (j - iscore_b[k])];



    isb = im_a * (sum_b + 1);
        
    dx = np.zeros(sum_b)
        
    isb = im_a * (sum_b + 1)
    
    print(dH)
    for j in range(sum_b):
            
        dx[j] = dH[isb + j + 1];
        msum += dx[j];
                
    result = list()
    for j in range(sum_b):
        result.append(dx[j]/msum)
    return result
    

In [ ]:
np.random.seed(3)
n = 3
A = np.asarray([np.random.randint(0,n,n,dtype=np.int32) for _ in range(5)])
B = np.asarray([np.random.randint(0,n,n,dtype=np.int32) for _ in range(5)])
#A = A_list[0]
#B = B_list[0]

In [ ]:
def GreenPvalFloat(pdist, A, n_samples, bins):
        """Calculate p-value for each sub-array
        Args:
            Z (array): The necessary part of the array to calculate sample p-values.
            n_samples (int): The total number of samples.
            S(int): Sum up to :m.
            A(array): The Values from sample A.
            bins(array): Bins for digitization.
        Returns:
            p-values
        """
        P = np.zeros(n_samples)
        for i, (a,b) in enumerate(zip(A,bins)):
            a_ = np.digitize(a, b).astype(np.int32) - 1
            p = np.sum(pdist[i,int(sum(a_)):])
            P[i] = 2 * min( p, (1-p))
                
        return P

def get_digitized_score(X, bins):
    """Digitize the values for each sample.
        Args:
            X (array): Concatenated sample from original samples A and B.
            bins(int): The number of bins to divide the sample values.
        Returns:
            digitized array
    """
    digitized = np.zeros(X.shape, dtype=np.int32)
    for i, (x,b) in enumerate(zip(X,bins)):
        digitized[i,:] = np.digitize(x, b).astype(np.int32) - 1
    return digitized
    
def _get_digitized_score(X, bins):
        """Digitize the values for each sample.
        Args:
            X (array): Concatenated sample from original samples A and B.
            bins(int): The number of bins to divide the sample values.
        Returns:
            digitized array
        """
        digitized = np.zeros(X.shape,dtype=np.int32)
        for i, (x,b) in enumerate(zip(X,bins)):
            digitized[i,:] = np.digitize(x, b).astype(np.int32) - 1
        return digitized
    
def GreenFloatDataPreProcess(A, B, num_bin):
        """Run method on the GPU.
        Args:
            A (array): Samples A.
            B (array): Samples B.
        Returns:
             p-values
        """
        
        m = A.shape[1]
        n = B.shape[1]

        n_samples = A.shape[0]

        X = np.concatenate([A,B],axis=1)
        X.sort()

        bins = np.asarray([np.linspace(np.min(x), np.max(x), int(num_bin)) for x in X])



        digitized = _get_digitized_score(X, bins)


        S = np.sum(digitized[:, m:], axis=1).astype(np.int32)
        return digitized.ravel(), m, n, S.max(), n_samples, bins

def GreenFloatCuda(A,B, num_bin, return_dperm=False):
    aDim, bDim = A.ndim,B.ndim
    assert aDim == bDim, "A and B does not have same dimensions!"
    
    if aDim == 1:
        A = A.reshape(1,-1)
        B = B.reshape(1,-1)
    a_n, b_n = A.shape[0], B.shape[0]
    assert a_n == b_n, "A and B does not have the same amount of experiments!"
    
    z, n, m, S, n_samples, bins =  run(A, B, num_bin)
    
    
        
    pdist = np.array(greenCUDAMultipleSamples(z, n, m, S, n_samples)).reshape(n_samples, S+1)
    

    
    p_values = GreenPvalFloat(pdist, A, n_samples, bins)
        
    if return_dperm:
        return p_values, pdist
    else:
        return p_values
    
    
    
        

In [ ]:
def GreenPvalInt(pdist, sums, n_samples):
    p_vals = np.zeros(n_samples)
    
    for i, s in enumerate(sums):
        p = pdist[i,s:].sum()
        p_vals[i] = 2 * min( p, (1-p))
    return p_vals

def GreenIntDataPreProcess(A,B):
    m = A.shape[1]
    n = B.shape[1]
    
    A_n_samples = A.shape[0]
    B_n_samples = B.shape[0]
    
    assert A_n_samples == B_n_samples, "A and B does not have same experiments!"
    n_samples = A_n_samples

    
    z = np.concatenate((A,B),axis=1)
    z.sort(1)
    z -= z.min(1, keepdims=True)
    S = max(z[:,m:].sum(1, keepdims=True))[0]
      
    return z.ravel(), m, n, S, n_samples


def GreenIntCuda(A,B, return_dperm=False):
    aDim, bDim = A.ndim,B.ndim
    assert aDim == bDim, "A and B does not have same dimensions!"
    
    if aDim == 1:
        A = A.reshape(1,-1)
        B = B.reshape(1,-1)
    a_n, b_n = A.shape[0], B.shape[0]
    assert a_n == b_n, "A and B does not have the same amount of experiments!"
    
    z, n, m, S, n_samples =  GreenIntDataPreProcess(A,B)
    

    pdist = np.array(greenCUDAMultipleSamples(z, n, m, S, n_samples)).reshape(n_samples, S+1)
    


    
    sums = (A - A.min(1, keepdims=True)).sum(1)
    
    p_values = GreenPvalInt(pdist, sums, n_samples)
        
    if return_dperm:
        return p_values, pdist
    else:
        return p_values
    
    
    
        

In [ ]:
k = 3

In [ ]:
%time pdist = GreenFloatCuda(A,B,3)

In [ ]:
pdist

In [ ]:
%time pdist = GreenIntCuda(A,B)

In [ ]:
pdist

In [ ]:

a, b = A[k], B[k]
%time p_valus = get_p(np.array(greenCUDA(*getDataGreen(a,b))),a)

In [ ]:
p_valus

In [ ]:
p_valus

In [ ]:
get_p(np.array(greenCUDA(*getDataGreen(A[4],B[4]))), A[0])

In [ ]:
p_valus

In [ ]:
get_p(p_dist,A[1])

In [ ]:
z.ravel()

In [ ]:
z.ravel().shape

In [ ]:
A.shape

In [ ]:
z = np.concatenate((A,B),axis=1)

In [ ]:
z.sort(1)

In [ ]:
z -= z.min(1, keepdims=True)

In [ ]:
S = max(z[:,2:].sum(1, keepdims=True))[0]

In [ ]:
S

In [ ]:
min(z)

In [ ]:
minVal = z.min(1, keepdims=True)

In [ ]:
z - minVal

In [ ]:
minVal.shape

In [ ]:
minVal

In [ ]:
z - minVal

In [ ]:
np.subtract(z,minVal, axis=1)

In [ ]:
z - minVal

In [ ]:
minVal.shape

In [ ]:
x1 = np.arange(9.0).reshape((3, 3))
x2 = np.arange(3.0)

In [ ]:
x1.shape

In [ ]:
x2.shape

In [ ]:
GreenCuda(A)

In [ ]:
z.min(1)

In [ ]:
getDataGreen(A,B)

In [ ]:
getDataCoinShift(A,B)

In [ ]:
x = coinPy(*getDataCoinShift(A,B))

In [ ]:
x

In [ ]:
coinShiftPdist(*getDataCoinShift(A,B))

In [ ]:
def shuffle(ix):
    random.shuffle(ix)
    return ix

In [ ]:
shuffle(list(range(20, min(2, 1)-1,-1)))

In [ ]:
all_time_coinC = list()
all_time_greenC = list()
all_time_greenMTC = list()
all_time_greenNumba = list()
all_time_coinR = list()
N = [50,100,150]
for n in N:
    time_coinC = list()
    time_greenC = list()
    time_greenMTC = list()
    time_greenNumba = list()
    time_coinR = list()

    A_list = np.asarray([np.random.randint(0,n,n,dtype=np.int32) for _ in range(1)])
    B_list = np.asarray([np.random.randint(0,n,n,dtype=np.int32) for _ in range(1)])
    for A,B in zip(A_list,B_list):
        start = time.time()
        d0 = np.array(coinShiftPdist(*getDataCoinShift(A,B)))
        end = time.time()
        time_coinC.append(end - start)

    
       

    
    
        start = time.time()
        d2 = np.array(GrennPdistOpenMP(*getDataGreen(A,B)))
        end = time.time()
        time_greenMTC.append(end - start)

    

    
    
        
        
    all_time_coinC.append(time_coinC)

    all_time_greenMTC.append(time_greenMTC)



    

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
mean_time_coinC = np.mean(all_time_coinC,1)

mean_time_greenMTC = np.mean(all_time_greenMTC,1)


In [ ]:
all_time_coinC

In [ ]:
plt.plot(N,mean_time_coinC, label="Coin C")

plt.plot(N,mean_time_greenMTC, label="GreenOpenMP C")


plt.legend(loc="upper left")
plt.xlabel("N")
plt.ylabel("Time(s)")